In [94]:
from Parser import Parser
from ipycanvas import Canvas
from typing import Dict, Callable, Tuple, List
import ipywidgets as widgets
import numpy as np
import math

MOVE_ACTION = 'move'
ZOOM_ACTION = 'zoom'
SELECT_ACTION = 'select'

def defaultColorMap(cells, variables):
    return [
        ( '#004400', 'green', cells[variables['cycle_model']] == 5),
        ('brown', 'black', cells[variables['cycle_model']] == 100),
        ( '#440000', 'red', cells[variables['cycle_model']] == 101)
    ]

def count(array):
    dic = {}
    for num in array:
        if not num in dic:
            dic[num] = 0
        dic[num] += 1
    return dic
    

class Interactor2D:
    def __init__(self, parser, width: int = 500, height: int = 400, colorMap = defaultColorMap):
        self._currentFrame = parser.getFrameRange()[0]
        self._canvas = Canvas(width=width, height=height)
        self._parser = parser
        self._colorMap = colorMap
        self._height = height
        self._width = width
        
        frame = parser.getFrame(self._currentFrame)
        mesh = frame.environment.mesh
        self._zoom = max((mesh.boundsX[1] - mesh.boundsX[0]) / width, (mesh.boundsY[1] - mesh.boundsY[0]) / height)
        
        self._xOffset = mesh.boundsX[0]
        self._yOffset = mesh.boundsY[0]
        self._selectedCell = None
        
        self._clicking = False
        self._dragStartX = 0
        self._dragStartY = 0
        self._actionOriginX = 0
        self._actionOriginY = 0
        
        self._canvas.on_mouse_down(self.onMouseDown)
        self._canvas.on_mouse_move(self.onMouseMove)
        self._canvas.on_mouse_up(self.onMouseUp)
        self._canvas.on_mouse_out(self.onMouseOut)
        
        self.action = MOVE_ACTION
        
        self._buttons = widgets.RadioButtons(
            options=[MOVE_ACTION, ZOOM_ACTION, SELECT_ACTION],
            value=self.action,
            description='Mouse Action:',
            disabled=False,
        )
        self._buttons.observe(self.onToolChange, names='value')
        
        self._frameSelector = widgets.IntSlider(
            value=self._currentFrame,
            min=parser.getFrameRange()[0],
            max=parser.getFrameRange()[1] - 1,
            step=1,
            description='Frame:',
            disabled=False,
            continuous_update=False,
            orientation='horizontal',
            readout=True,
            readout_format='d'
        )
        self._frameSelector.observe(self.onFrameChange, names='value')
        
        self.update()
    
    def update(self):
        self._canvas.clear()
        
        canvas = self._canvas
        
        frame = self._parser.getFrame(self._currentFrame)
    
        cells = frame.cells.data
        cellVariables = frame.cells.variables
        
        canvas.stroke_style = 'blue'

        x = (cells[cellVariables['position.x']] - self._xOffset) / self._zoom
        y = (cells[cellVariables['position.y']] - self._yOffset) / self._zoom
        r = self.radiusOfCells(cells, cellVariables) / self._zoom
        
        combined = np.array([x, y, r])
        
        for fill, stroke, indices in self._colorMap(cells, cellVariables):
            split = combined[:,indices]
            if split.shape[0] == 0:
                continue
            x, y, r = split
            canvas.fill_style = fill
            canvas.fill_circles(x, y, r)
            canvas.stroke_style = stroke
            canvas.stroke_circles(x, y, r)
        
        canvas.fill_style = '#A0A0A0'
        canvas.font = '10px serif'
        canvas.fill_text(f"(x: {self._xOffset}, y: {self._yOffset}, s: {self._zoom}, c: {self._selectedCell})", 10, self._height - 10)
        
    
    def onMouseDown(self, x: int, y: int):
        self._dragStartX = x
        self._dragStartY = y
        self._envActionOriginX = x * self._zoom + self._xOffset
        self._envActionOriginY = y * self._zoom + self._yOffset
        self._actionOriginX = x
        self._actionOriginY = y
        self._actionOriginZoom = self._zoom
        
        if self.action == SELECT_ACTION:
            self.selectCell(self._envActionOriginX, self._envActionOriginY)
        else:
            self._clicking = True
    
    def onMouseUp(self, x: int, y: int):
        self._clicking = False
    
    def onMouseOut(self, x: int, y: int):
        self._clicking = False
    
    def onMouseMove(self, x: int, y: int):
        if self._clicking:
            if self.action == MOVE_ACTION:
                self._xOffset -= (x - self._dragStartX) * self._zoom
                self._yOffset -= (y - self._dragStartY) * self._zoom
            elif self.action == ZOOM_ACTION:
                self._zoom = max(self._actionOriginZoom * 2 ** ((self._actionOriginY - y) / 25), 0.0001)
                self._xOffset = self._envActionOriginX - self._actionOriginX * self._zoom
                self._yOffset = self._envActionOriginY - self._actionOriginY * self._zoom
                
            self._dragStartX = x
            self._dragStartY = y
            self.update()
    
    def onToolChange(self, action):
        self.action = action.new
    
    def onFrameChange(self, action):
        self._currentFrame = action.new
        self.update()
    
    def selectCell(self, x: float, y: float):
        frame = self._parser.getFrame(self._currentFrame)
    
        cells = frame.cells.data
        cellVariables = frame.cells.variables
        
        distances = np.sqrt(np.square(x - cells[cellVariables['position.x']]) + np.square(y - cells[cellVariables['position.y']])) - self.radiusOfCells(cells, cellVariables)
        minIndex = np.argmin(distances)
        
        if distances[minIndex] <= 0:
            self._selectedCell = cells[cellVariables['ID'], minIndex]
            self.update()
        
        
 
    def radiusOfCells(self, cells, variables):
        return (cells[variables['total_volume']] * (3 / ( 4 * math.pi))) ** (1 / 3)

    def show(self):
        display(self._canvas, self._buttons, self._frameSelector)
            



def InteractiveEnvironment(outputPath: str, width: int = 500, height: int = 400):
    parser = Parser(outputPath)
    if parser.getFrame(parser.getFrameRange()[0]).environment.is2D:
        return Interactor2D(parser, width, height)

In [95]:
env = InteractiveEnvironment('./sample-output', width=800, height=800)
env.show()

Canvas(height=800, width=800)

RadioButtons(description='Mouse Action:', options=('move', 'zoom', 'select'), value='move')

IntSlider(value=0, continuous_update=False, description='Frame:', max=9)

In [96]:
help(Canvas)

Help on class Canvas in module ipycanvas.canvas:

class Canvas(_CanvasBase)
 |  Canvas(*args, **kwargs)
 |  
 |  Create a Canvas widget.
 |  
 |  Args:
 |      width (int): The width (in pixels) of the canvas
 |      height (int): The height (in pixels) of the canvas
 |      caching (boolean): Whether commands should be cached or not
 |  
 |  Method resolution order:
 |      Canvas
 |      _CanvasBase
 |      ipywidgets.widgets.domwidget.DOMWidget
 |      ipywidgets.widgets.widget.Widget
 |      ipywidgets.widgets.widget.LoggingHasTraits
 |      traitlets.traitlets.HasTraits
 |      traitlets.traitlets.HasDescriptors
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, *args, **kwargs)
 |      Create a Canvas widget.
 |  
 |  __setattr__(self, name, value)
 |      Implement setattr(self, name, value).
 |  
 |  arc(self, x, y, radius, start_angle, end_angle, anticlockwise=False)
 |      Add a circular arc centered at ``(x, y)`` with a radius of ``radius`` to th

In [181]:
env.show()

Canvas(height=800, width=800)

RadioButtons(description='Mouse Action:', options=('move', 'zoom', 'select'), value='move')

IntSlider(value=0, continuous_update=False, description='Frame:', max=9)

In [100]:
math.sig(-1)

AttributeError: module 'math' has no attribute 'sig'

In [103]:
int(False) * 2 - 1

-1